# Data Cleaning

## Import Relevant Packages and Libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandoc

In [3]:
pd.set_option('display.max_columns', None)

## Import the data

In [4]:
# read in the data
df_insta = pd.read_csv('../data/raw/Post Performance (Stanbic IBTC) January 1, 2013 - July 13, 2023_Instagram.csv')
df_lnkd = pd.read_csv('../data/raw/Post Performance (Stanbic IBTC) January 1, 2013 - July 13, 2023_Linkedin.csv')
df_twit = pd.read_csv('../data/raw/Post Performance (Stanbic IBTC) January 1, 2013 - July 13, 2023_Twitter.csv')
df_fbk = pd.read_csv('../data/raw/Post Performance (Stanbic IBTC) January 1, 2013 - July 13, 2023_Facebook.csv')

C:\Users\Victor Essien\AppData\Local\Temp\ipykernel_2536\2350594635.py:2: DtypeWarning: Columns (21,22,30,31,32,33,40,78,80) have mixed types. Specify dtype option on import or set low_memory=False.
  df_insta = pd.read_csv('../data/raw/Post Performance (Stanbic IBTC) January 1, 2013 - July 13, 2023_Instagram.csv')
C:\Users\Victor Essien\AppData\Local\Temp\ipykernel_2536\2350594635.py:3: DtypeWarning: Columns (78,80) have mixed types. Specify dtype option on import or set low_memory=False.
  df_lnkd = pd.read_csv('../data/raw/Post Performance (Stanbic IBTC) January 1, 2013 - July 13, 2023_Linkedin.csv')
C:\Users\Victor Essien\AppData\Local\Temp\ipykernel_2536\2350594635.py:4: DtypeWarning: Columns (32,33,41,53,61,78,146) have mixed types. Specify dtype option on import or set low_memory=False.
  df_twit = pd.read_csv('../data/raw/Post Performance (Stanbic IBTC) January 1, 2013 - July 13, 2023_Twitter.csv')
C:\Users\Victor Essien\AppData\Local\Temp\ipykernel_2536\2350594635.py:5: DtypeW

## First Look

In [5]:
# function to get the shape of all dataframes
def get_shape(*dataframes):
    for idx, df in enumerate(dataframes, start=1):
        print(f"Shape of {df['Network'].values[0]} dataset: {df.shape}")
    

In [6]:
get_shape(df_insta, df_lnkd, df_twit, df_fbk)

Shape of Instagram dataset: (10000, 147)
Shape of LinkedIn dataset: (7760, 147)
Shape of Twitter dataset: (8529, 147)
Shape of Facebook dataset: (9803, 147)


They all have the same number of columns, but are they the same?

In [7]:
# check if all dataframes have the same columns
if list(df_insta.columns) == list(df_twit.columns) == list(df_lnkd.columns) == list(df_fbk.columns):
   print('True')
else: 
    print('False')

True


Since they all have the same columns, a function will be needed to clean all of them. Before that the columns need to be investigated to determine what impurities or inconsistencies exist within the data, so as to inform cleaning steps.

## Cleaning

To clean all dataframes without concatenating them, I create a test dataframe, scope the columns for impurities and inconsistencies then define a function to generalize to the four dataframes.

### Defining the cleaning process:
- Standardize column names
- Set appropriate data types
- Check for general inconsistencies
- Check for and deal with missing values
- Remove duplicates
- Remove excessive whitespace from string columns
- Define function and Save clean data to a csv file

#### Standardize column names

In [8]:
# make a copy of one dataset
df_test = df_insta.copy()

In [9]:
# view columns
df_test.columns

Index(['Date', 'Post ID', 'Network', 'Post Type', 'Content Type', 'Profile',
       'Sent by', 'Link', 'Post', 'Linked Content',
       ...
       'Video Removed from Playlists', 'Annotation Impressions',
       'Annotation Clickable Impressions', 'Annotation Closable Impressions',
       'Annotation Closes', 'Card Impressions', 'Card Teaser Impressions',
       'Card Teaser Clicks', 'Poll Votes', 'Tags'],
      dtype='object', length=147)

In [10]:
# replace spaces and special character with '_'
df_test.columns = df_test.columns.str.lower().str.replace('-', '_').str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

#### Set appropriate data types

To do this, the columns need to be checked to understand why pandas could not define their data types on import or what data types are mixed within such columns.

In [11]:
df_test.dtypes

date                        object
post_id                      int64
network                     object
post_type                   object
content_type                object
                            ...   
card_impressions           float64
card_teaser_impressions    float64
card_teaser_clicks         float64
poll_votes                 float64
tags                        object
Length: 147, dtype: object

From the output above of the data types of each column, it can be seen already that some columns dont have the correct data type. This may be due to inconsistencies of the data within those columns. So those columns need to be cleaned and set to the appropriate data type. An example is the date column.

A further look can be had at the data type of the first 20 columns below:

In [12]:
df_test.dtypes[:20]

date                            object
post_id                          int64
network                         object
post_type                       object
content_type                    object
profile                         object
sent_by                         object
link                            object
post                            object
linked_content                 float64
impressions                     object
organic_impressions             object
viral_impressions              float64
non_viral_impressions          float64
paid_impressions               float64
fan_impressions                float64
fan_organic_impressions        float64
fan_paid_impressions           float64
non_fan_impressions            float64
non_fan_organic_impressions    float64
dtype: object

The impressions columns should contain only numbers.

In [13]:
 # convert date column to datetime
df_test['date'] = pd.to_datetime(df_test['date'], format='%m/%d/%Y %I:%M %p')
       
# convert string columns with numbers to float
for column in df_test.columns:
    if df_test[column].dtype == 'object':
        if df_test[column].str.isnumeric().any():
            df_test[column] = df_test[column].str.replace(',','').astype(float)
        elif 'per' in df_test[column].name:
            df_test[column] = df_test[column].str.replace(',','').str.replace('%','').astype(float)
        

In [14]:
# check to confirm 
df_test.dtypes[:20]

date                           datetime64[ns]
post_id                                 int64
network                                object
post_type                              object
content_type                           object
profile                                object
sent_by                                object
link                                   object
post                                   object
linked_content                        float64
impressions                           float64
organic_impressions                   float64
viral_impressions                     float64
non_viral_impressions                 float64
paid_impressions                      float64
fan_impressions                       float64
fan_organic_impressions               float64
fan_paid_impressions                  float64
non_fan_impressions                   float64
non_fan_organic_impressions           float64
dtype: object

#### Check for general inconsistencies

While scoping some of the categorical columns, something curious was discovered. See below:

In [15]:
df_test.sent_by.value_counts()

sent_by
                     4967
Blessing Ubah        2707
Aramide Salami       1733
Kemi Amoo             315
Sophia Amuka          192
Damilare Oyekanmi      86
Name: count, dtype: int64

There are 4967 post made by ' '. There are too many rows to just drop and those rows contain important data from which insights can be extracted. The best thing is to replace that white space with 'admin'. The reasoning is someone was making those posts on behalf of the profile.

In [16]:
# replace white space in sent_by with admin
df_test['sent_by'] = df_test['sent_by'].replace(' ', 'admin')

In [17]:
# to confirm
df_test.sent_by.value_counts()

sent_by
admin                4967
Blessing Ubah        2707
Aramide Salami       1733
Kemi Amoo             315
Sophia Amuka          192
Damilare Oyekanmi      86
Name: count, dtype: int64

#### Check for and deal with missing values

In [18]:
# check percentage of missing values per column
missing_percentage = df_test.isnull().sum()/len(df_test)*100
missing_types = df_test.dtypes
missing_values = pd.DataFrame({'missing_percentage' : missing_percentage, 'dtype' : missing_types})

In [19]:
# missing_values.missing_percentage *10000
missing_values[:20]

,missing_percentage,dtype
date,0.00,datetime64[ns]
post_id,0.00,int64
network,0.00,object
post_type,0.00,object
content_type,0.00,object
profile,0.00,object
sent_by,0.00,object
link,0.00,object
post,0.11,object
linked_content,100.00,float64


In [20]:
df_test[df_test.post.isnull()]

,date,post_id,network,post_type,content_type,profile,sent_by,link,post,linked_content,impressions,organic_impressions,viral_impressions,non_viral_impressions,paid_impressions,fan_impressions,fan_organic_impressions,fan_paid_impressions,non_fan_impressions,non_fan_organic_impressions,non_fan_paid_impressions,reach,organic_reach,viral_reach,non_viral_reach,paid_reach,fan_reach,fan_paid_reach,potential_reach,engagement_rate_per_impression,engagement_rate_per_reach,engagements,reactions,likes,dislikes,love_reactions,haha_reactions,wow_reactions,sad_reactions,angry_reactions,comments,shares,saves,click_through_rate,post_link_clicks,other_post_clicks,post_clicks_all,post_media_clicks,post_hashtag_clicks,post_detail_expand_clicks,profile_clicks,post_photo_view_clicks,post_video_play_clicks,other_engagements,answers,app_engagements,app_install_attempts,app_opens,follows_from_post,unfollows_from_post,negative_feedback,bit.ly_link_clicks,engaged_users,engaged_fans,users_talking_about_this,unique_reactions,unique_comments,unique_shares,unique_answers,unique_post_clicks,unique_post_link_clicks,unique_post_photo_view_clicks,unique_post_video_play_clicks,unique_other_post_clicks,unique_negative_feedback,subscribers_gained_from_video,annotation_clicks,card_clicks,video_views,media_views,organic_video_views,paid_video_views,partial_video_views,organic_partial_video_views,paid_partial_video_views,full_video_views,full_video_view_rate,follow_video_views,for_you_video_views,hashtag_video_views,business_account_video_views,sound_video_views,unspecified_video_views,organic_full_video_views,paid_full_video_views,autoplay_video_views,click_to_play_video_views,sound_on_video_views,sound_off_video_views,10_second_video_views,organic_10_second_video_views,paid_10_second_video_views,autoplay_10_second_video_views,click_to_play_10_second_video_views,sound_on_10_second_video_views,sound_off_10_second_video_views,autoplay_partial_video_views,click_to_play_partial_video_views,autoplay_full_video_views,click_to_play_full_video_views,95%_video_views,organic_95%_video_views,paid_95%_video_views,video_length_seconds,average_video_time_watched_seconds,video_view_time_seconds,organic_video_view_time_seconds,paid_video_view_time_seconds,unique_video_views,unique_organic_video_views,unique_paid_video_views,unique_10_second_video_views,unique_full_video_views,unique_organic_95%_video_views,unique_paid_95%_video_views,video_ad_break_ad_impressions,video_ad_break_ad_earnings,video_ad_break_ad_cost_per_impression_cpm,youtube_premium_views,estimated_minutes_watched,estimated_premium_minutes_watched,story_taps_back,story_taps_forward,story_exits,story_replies,video_added_to_playlists,subscribers_lost_from_video,video_removed_from_playlists,annotation_impressions,annotation_clickable_impressions,annotation_closable_impressions,annotation_closes,card_impressions,card_teaser_impressions,card_teaser_clicks,poll_votes,tags
1423,2021-10-06 15:18:00,18264735889006746,Instagram,Post,Video,stanbicibtc,admin,https://www.instagram.com/tv/CUsLOdsoeK7/,NaN,NaN,3024.0,3024.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2681.0,2681.0,NaN,NaN,NaN,NaN,NaN,NaN,3.31,3.73,100.0,82.0,82.0,NaN,NaN,NaN,NaN,NaN,NaN,16.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1673.0,NaN,1673.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Reward4Saving
2687,2022-11-07 15:15:00,17996455921508611,Instagram,Post,Video,stanbicibtc,admin,https://www.instagram.com/tv/Ckqad7iI4Lx/,NaN,NaN,2195.0,2195.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1982.0,1982.0,NaN,NaN,NaN,NaN,NaN,NaN,3.51,3.88,77.0,63.0,63.0,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [21]:
df_test[df_test.impressions.isnull()]

,date,post_id,network,post_type,content_type,profile,sent_by,link,post,linked_content,impressions,organic_impressions,viral_impressions,non_viral_impressions,paid_impressions,fan_impressions,fan_organic_impressions,fan_paid_impressions,non_fan_impressions,non_fan_organic_impressions,non_fan_paid_impressions,reach,organic_reach,viral_reach,non_viral_reach,paid_reach,fan_reach,fan_paid_reach,potential_reach,engagement_rate_per_impression,engagement_rate_per_reach,engagements,reactions,likes,dislikes,love_reactions,haha_reactions,wow_reactions,sad_reactions,angry_reactions,comments,shares,saves,click_through_rate,post_link_clicks,other_post_clicks,post_clicks_all,post_media_clicks,post_hashtag_clicks,post_detail_expand_clicks,profile_clicks,post_photo_view_clicks,post_video_play_clicks,other_engagements,answers,app_engagements,app_install_attempts,app_opens,follows_from_post,unfollows_from_post,negative_feedback,bit.ly_link_clicks,engaged_users,engaged_fans,users_talking_about_this,unique_reactions,unique_comments,unique_shares,unique_answers,unique_post_clicks,unique_post_link_clicks,unique_post_photo_view_clicks,unique_post_video_play_clicks,unique_other_post_clicks,unique_negative_feedback,subscribers_gained_from_video,annotation_clicks,card_clicks,video_views,media_views,organic_video_views,paid_video_views,partial_video_views,organic_partial_video_views,paid_partial_video_views,full_video_views,full_video_view_rate,follow_video_views,for_you_video_views,hashtag_video_views,business_account_video_views,sound_video_views,unspecified_video_views,organic_full_video_views,paid_full_video_views,autoplay_video_views,click_to_play_video_views,sound_on_video_views,sound_off_video_views,10_second_video_views,organic_10_second_video_views,paid_10_second_video_views,autoplay_10_second_video_views,click_to_play_10_second_video_views,sound_on_10_second_video_views,sound_off_10_second_video_views,autoplay_partial_video_views,click_to_play_partial_video_views,autoplay_full_video_views,click_to_play_full_video_views,95%_video_views,organic_95%_video_views,paid_95%_video_views,video_length_seconds,average_video_time_watched_seconds,video_view_time_seconds,organic_video_view_time_seconds,paid_video_view_time_seconds,unique_video_views,unique_organic_video_views,unique_paid_video_views,unique_10_second_video_views,unique_full_video_views,unique_organic_95%_video_views,unique_paid_95%_video_views,video_ad_break_ad_impressions,video_ad_break_ad_earnings,video_ad_break_ad_cost_per_impression_cpm,youtube_premium_views,estimated_minutes_watched,estimated_premium_minutes_watched,story_taps_back,story_taps_forward,story_exits,story_replies,video_added_to_playlists,subscribers_lost_from_video,video_removed_from_playlists,annotation_impressions,annotation_clickable_impressions,annotation_closable_impressions,annotation_closes,card_impressions,card_teaser_impressions,card_teaser_clicks,poll_votes,tags
8516,2023-06-12 13:08:00,18029321227525834,Instagram,Post,Carousel,stanbicibtc,Blessing Ubah,https://www.instagram.com/p/CtY8c1Ts0pP/,"At Stanbic IBTC, we hold special gatherings fo...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"BCC, Enterprise Academy Monday"
8517,2023-06-08 09:55:00,18365455210048288,Instagram,Post,Photo,stanbicibtc,Blessing Ubah,https://www.instagram.com/p/CtOTPwcMPZ-/,Motor insurance is required for any ‘Idan’ who...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

Observing the frames above, for both categorical and numerical columns dropping those missing values doesn't make much sense. For the numerical columns however, filling with zero should be done. This will be done during EDA.

#### Remove duplicates

In [22]:
# check for and remove duplicates
if df_test.duplicated().sum() > 0:
    df_test.drop_duplicates(inplace=True)

#### Remove excessive whitespace from string columns

In [23]:
for column in df_test.columns:
    if df_test[column].dtype == 'object':
        df_test[column] = df_test[column].str.strip()

#### Define function and Save clean data to a csv file

In [24]:
def clean_data(*dataframes):
    """
    Cleans and standardizes a list of DataFrames and saves them to CSV files.

    Parameters:
    *dataframes (pd.DataFrame): Variable number of input DataFrames to be cleaned.

    Notes:
    - Column names are converted to lowercase and special characters are replaced with underscores.
    - The 'date' column is converted to a datetime format.
    - Columns containing numeric strings are converted to float, handling comma and percent formatting.
    - 'sent_by' column is corrected by replacing white spaces with 'admin'.
    - Duplicate rows are dropped if any exist.
    - Each cleaned DataFrame is saved to a separate CSV file with a unique name.
    """
    
    for i, dataframe in enumerate(dataframes, 1):
        # standardize column names
        dataframe.columns = dataframe.columns.str.lower()\
        .str.replace('-', '_')\
        .str.replace(' ', '_')\
        .str.replace('(', '')\
        .str.replace(')', '')
        
        # convert date column to datetime
        dataframe['date'] = pd.to_datetime(dataframe['date'], format='%m/%d/%Y %I:%M %p')
       
        # convert string columns with numbers to float
        for column in dataframe.columns:
            if dataframe[column].dtype == 'object':
                if dataframe[column].str.isnumeric().any():
                    dataframe[column] = dataframe[column].str.replace(',','').astype(float)
                elif 'per' in dataframe[column].name:
                    dataframe[column] = dataframe[column].str.replace(',','').str.replace('%','').astype(float)
                elif 'rate' in dataframe[column].name:
                    dataframe[column] = dataframe[column].str.replace(',','').str.replace('%','').astype(float)
        
        # replace white space in sent_by with admin
        dataframe['sent_by'] = dataframe['sent_by'].replace(' ', 'admin')
        
        # drop duplicates if any exist
        if dataframe.duplicated().sum() > 0:
            dataframe.drop_duplicates(inplace=True)
        
        # remove trailing whitespace from string columns
        for column in dataframe.columns:
            if dataframe[column].dtype == 'object':
                dataframe[column] = dataframe[column].str.strip()
        
        # Save the cleaned DataFrame to a CSV file with a unique name
        csv_filename = f"../data/processed/Cleaned-Post Performance (Stanbic IBTC) January 1, 2013 - July 13, 2023_{dataframe['network'].values[0]}.csv"
        dataframe.to_csv(csv_filename, index=False)
        
            
    

In [25]:
clean_data(df_insta, df_lnkd, df_twit, df_fbk)